# ABS Quarterly International Trade 5302

## Python set-up

In [1]:
# analytic imports
import pandas as pd
import matplotlib.pyplot as plt
import readabs as ra
from readabs import metacol as mc

# local imports
from abs_helper import get_abs_data, get_gdp
from plotting import line_plot

# pandas display settings
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 999

# display charts in this notebook
SHOW = False

## Get data from ABS

In [2]:
abs_dict, meta, source, RECENT = get_abs_data("5302.0")
plot_times = None, RECENT
plot_tags = "", "-recent"

## Plot

### Headline charts

In [3]:
def headline_charts() -> None:
    """Produce headline charts."""

    # special cases
    table = "530204"
    wanted = {
        "Current Account Balance": "Current account ;",
        "Balance of Trade in Goods and Services": "Goods and Services ;",
        "Net Primary Income": "Primary income ;",
        "Net Secondary Income": "Secondary income ;",
    }

    def plotme(series, title, units, **kwargs):
        rfooter = f"{source} {table} {kwargs.get('rfooter', '')}"
        line_plot(
            series, 
            starts=plot_times,
            tags=plot_tags,
            title=title,
            ylabel=units,
            y0=True,
            rfooter=rfooter,
            lfooter="Australia. Current Prices. ",
            show=SHOW,
        )

    box = {}
    gdp, gdp_units = get_gdp("CP", "SA")
    gdp, gdp_units = ra.recalibrate(gdp, gdp_units)
    expected_units = "billion"
    assert expected_units in gdp_units.lower(), f"Expected units: {expected_units}, got: {gdp_units}"
    stype = "Seasonally Adjusted"
    for title, handle in wanted.items():
        selector = {stype: mc.stype, handle: mc.did, table: mc.table}
        _table, series_id, units = ra.find_abs_id(meta, selector, exact_match=True)
        series, units = ra.recalibrate(abs_dict[_table][series_id], units)
        assert expected_units in units.lower(), f"Expected units: {expected_units}, got: {units}"
        box[title] = [series, units]
        plotme(series, title, units)
        plotme(series / gdp * 100, title + " as % of GDP", "Per cent", rfooter="5206.0")
    
    net_income = box["Net Primary Income"][0] + box["Net Secondary Income"][0]
    units = box["Net Primary Income"][1]
    assert units == box["Net Secondary Income"][1]
    plotme(net_income, "Net Income", units)
    plotme(net_income / gdp * 100, "Net Income as % of GDP", "Per cent", rfooter="5206.0")

    # sanity check
    cad = box.pop("Current Account Balance")[0]
    components = pd.DataFrame({k: v[0] for k, v in box.items()})
    assert ((cad - components.sum(axis=1)) < 0.01).all() 


headline_charts()

## Finished

In [4]:
# watermark
%load_ext watermark
%watermark --python --conda --machine --iversions --watermark

Python implementation: CPython
Python version       : 3.12.8
IPython version      : 8.31.0

conda environment: 312

Compiler    : Clang 18.1.8 
OS          : Darwin
Release     : 24.2.0
Machine     : arm64
Processor   : arm
CPU cores   : 14
Architecture: 64bit

pandas    : 2.2.3
matplotlib: 3.10.0
readabs   : 0.0.20

Watermark: 2.5.0



In [5]:
print("Finished")

Finished
